In [5]:
import sys
sys.path.append("../h2o-py/build/main")
import h2o
import time
import os
import statistics
import random

In [6]:
def get_folder_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size


def to_MB(size_in_bytes):
    return size_in_bytes / 1024**2



def measure_model(ntrees, max_depth, checkpoints_path=None):
    times = []
    file_sizes = []
    
    for seed in range(0,10):
        if checkpoints_path!=None: 
            checkpoints_path_unique = checkpoints_path + "_" + str(ntrees) + "_" + str(max_depth) + "_" + str(seed)
            print("Checkpoints path:", checkpoints_path_unique)
        else:
            checkpoints_path_unique=None
            
        creditcards_gbm = H2OGradientBoostingEstimator(ntrees=ntrees,
                                            max_depth=max_depth,
                                            min_rows=1,
                                            score_tree_interval=ntrees+1,
                                            in_training_checkpoints_dir=checkpoints_path_unique,
                                            in_training_checkpoints_tree_interval=5,           
                                            seed = seed)
        start = time.time()
        creditcards_gbm.train(x=predictors,
                       y=response,
                       training_frame=train)
        end = time.time()
        training_time = end - start
        print("Took", training_time, "s")
        times.append(training_time)
        model_path = h2o.save_model(creditcards_gbm, "models/")
        file_stats = os.stat(model_path)
        model_size = to_MB(file_stats.st_size)
        file_sizes.append(model_size)
        
    print("Mean training times: ", statistics.mean(times))
    print("Mean model size in MegaBytes is", statistics.mean(file_sizes))

    if checkpoints_path != None:
        checkpoints_folder_size = to_MB(get_folder_size(f"{checkpoints_path_unique}"))
        print("Size of all checkpoints is: ", checkpoints_folder_size, 'MB')
        print("Its ", checkpoints_folder_size / statistics.mean(file_sizes), "times more that size of final model")
        
    return times, file_sizes

In [7]:
from h2o.estimators import H2OGradientBoostingEstimator
h2o.init()

creditcards = h2o.import_file("../bigdata/laptop/creditcardfraud/creditcardfraud.csv")

columns = creditcards.columns
predictors = columns[:len(columns)-1]
response = "Class"

creditcards[response] = creditcards[response].asfactor()

train = creditcards

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 min 37 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.37.0.99999
H2O_cluster_version_age:,15 minutes
H2O_cluster_name:,H2O_from_python_adam_farajn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.419 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
slowdowns = []
for ntrees in [100, 200, 500, 1000, 1500, 2000]:
    model = measure_model(ntrees, 30)
    model_checkpoints = measure_model(ntrees, 30, "checkpoints_gbm")
    display("Without checkpoints times", model)
    display("With checkpoints times", model_checkpoints)
    slowdown = statistics.mean(model_checkpoints[0])/statistics.mean(model[0]) - 1
    slowdowns.append(slowdown)
    print("Training time is:", slowdown*100, "% different with checkpoints")
    
display("Slowdowns", slowdowns)
print("Training time slowdown average is:", statistics.mean(slowdowns))

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 29.94912886619568 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 31.235809087753296 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 45.05717706680298 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 43.106624126434326 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 42.68444895744324 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 42.672333002090454 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 43.299466133117676 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 42.150651931762695 s
gbm Model Build progress: |████████

'Without checkpoints times'

([29.94912886619568,
  31.235809087753296,
  45.05717706680298,
  43.106624126434326,
  42.68444895744324,
  42.672333002090454,
  43.299466133117676,
  42.150651931762695,
  42.472980260849,
  43.61157774925232],
 [1.4884777069091797,
  1.4884719848632812,
  1.4884872436523438,
  1.4884729385375977,
  1.4884843826293945,
  1.488480567932129,
  1.4884605407714844,
  1.488480567932129,
  1.4884653091430664,
  1.4884843826293945])

'With checkpoints times'

([42.49516415596008,
  45.32595109939575,
  41.53921413421631,
  44.399133920669556,
  41.70758414268494,
  42.87638330459595,
  39.652929067611694,
  40.71227812767029,
  41.3424129486084,
  39.98939394950867],
 [1.4885263442993164,
  1.488530158996582,
  1.488530158996582,
  1.488525390625,
  1.4885225296020508,
  1.4885120391845703,
  1.4885358810424805,
  1.4885292053222656,
  1.488515853881836,
  1.488530158996582])

...

([291.76204681396484,
  297.0929400920868,
  294.62653613090515,
  295.4833097457886,
  294.1173839569092,
  295.11101484298706,
  294.3508548736572,
  295.26317715644836,
  296.12681007385254,
  294.61343908309937],
 [6.22425651550293,
  6.224198341369629,
  6.224237442016602,
  6.22414493560791,
  6.22419548034668,
  6.224128723144531,
  6.224124908447266,
  6.224221229553223,
  6.22419548034668,
  6.224289894104004])

Training time is: 1.4296793992224055 % different with checkpoints


'Slowdowns'

[0.03397066012905525,
 0.003888111782581083,
 -0.04901806390228469,
 -0.024449903308654708,
 0.0069537012271032594,
 0.014296793992224055]

Training time slowdown average is: -0.0023931166799959582
